In [ ]:
!pip install transformers

In [ ]:
!pip install hazm

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

import re
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import decomposition
from sklearn.svm import LinearSVC, NuSVC, SVC
from sklearn.metrics import f1_score, accuracy_score, hamming_loss
from sklearn.pipeline import Pipeline
from sklearn.multiclass import OneVsRestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, HashingVectorizer
from scipy import linalg
import networkx as nx
from collections import Counter, defaultdict
import pickle
import nltk
nltk.download('wordnet')
from nltk import stem
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from gensim import matutils, models
from gensim.models import Word2Vec
from wordcloud import WordCloud
import scipy.sparse
from wordcloud import WordCloud, STOPWORDS
import seaborn as sns
from nltk import word_tokenize
import warnings
warnings.filterwarnings('ignore')
! pip install apyori
from apyori import apriori
from mlxtend.preprocessing import TransactionEncoder
from multiprocessing import  Pool
import transformers
import tensorflow as tf
from tqdm import tqdm

from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding,LSTM,Dense,SpatialDropout1D
from keras.optimizers import Adam
from keras.initializers import Constant

from hazm import *

plt.rcParams['figure.figsize'] = [24, 12]
plt.style.use('seaborn-darkgrid')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy()

print("REPLICAS available: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.122.123.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.122.123.90:8470


INFO:tensorflow:Initializing the TPU system: grpc://10.122.123.90:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


REPLICAS available:  8


In [ ]:
from google.colab import  drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/digimag/

/content/drive/MyDrive/digimag


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/digimag/digimag/train.csv', sep='\t', encoding="utf-8")
train.columns = train.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print('Train Data shape: ', train.shape)
train.head(5)

Train Data shape:  (6896, 4)


,unnamed:_0,content,label,label_id
0,0,نمایش تبلیغ در لاک‌اسکرین تعدادی از گوشی‌های ه...,علم و تکنولوژی,3
1,1,شکست Justice League در باکس آفیس پس از بازخورد...,هنر و سینما,5
2,2,کلاسیک بینی؛ همه چیز در یک شب اتفاق افتاد فیلم...,هنر و سینما,5
3,3,اپل دوباره سراغ رنده رفته چراکه آپگرید کردن سط...,علم و تکنولوژی,3
4,4,بررسی جزء به جزء بهترین بخش Ori and the Blind ...,بازی ویدیویی,0


In [ ]:
test = pd.read_csv('/content/drive/MyDrive/digimag/digimag/test.csv', sep='\t', encoding="utf-8")
test.columns = test.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
print('Test Data shape: ', test.shape)
test.head(5)

Test Data shape:  (852, 4)


,unnamed:_0,content,label,label_id
0,0,با این ۵ صبحانه رژیمی لاغر شوید صبحانه همیشه ب...,سلامت و زیبایی,2
1,1,نمایشگر گوشی تاشونده ال‌جی مانند بروشور باز و ...,علم و تکنولوژی,3
2,2,چرا برخی سلبریتی‌های هالیوودی از شبکه‌های اجتم...,هنر و سینما,5
3,3,گوگل استیدیا؛ آینده‌ی صنعت بازی یا محکوم به شک...,بازی ویدیویی,0
4,4,ویژگی جذاب گلکسی Z Flip به زودی به گوشی‌های ان...,علم و تکنولوژی,3


In [ ]:
train['label'].value_counts()

علم و تکنولوژی    2245
بازی ویدیویی      1593
هنر و سینما       1350
سلامت و زیبایی    1304
کتاب و ادبیات      206
راهنمای خرید       101
عمومی               97
Name: label, dtype: int64

In [ ]:
import nltk
nltk.download("stopwords")

def remove_stopwords(string):
    word_list = [word.lower() for word in string.split()]
    stopwords_list = list(stopwords.words("english"))
    for word in word_list:
        if word in stopwords_list:
            word_list.remove(word)
    return ' '.join(word_list)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
sentence_train = train['content']
label_train = train['label_id']

print('Number of training sentence: ', sentence_train.shape)
print('Number of training label: ', label_train.shape)

Number of training sentence:  (6896,)
Number of training label:  (6896,)


In [ ]:
train.label_id.value_counts()

3    2245
0    1593
5    1350
2    1304
6     206
1     101
4      97
Name: label_id, dtype: int64

In [ ]:
train.isnull().sum()

unnamed:_0    0
content       0
label         0
label_id      0
dtype: int64

In [ ]:
test.isnull().sum()

unnamed:_0    0
content       0
label         0
label_id      0
dtype: int64

In [ ]:
from collections import Counter
cnt = Counter(label_train)
cnt = dict(cnt)
print(cnt)

{3: 2245, 5: 1350, 0: 1593, 6: 206, 2: 1304, 4: 97, 1: 101}


In [ ]:
from tensorflow.keras.utils import to_categorical

sentence_train = np.asarray(sentence_train)
label_train = np.asarray(label_train)
categorical_label_train = to_categorical(label_train, 7)
categorical_label_train

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [ ]:
import re
normalizer = Normalizer()
lemmatizer = Lemmatizer()
stemmer = Stemmer()

def clean_data(doc):
    doc = normalizer.normalize(doc) 
    tokenized = word_tokenize(doc)  
    tokens = []
    for token in tokenized:      
      token = re.sub("[،:.,;()/+]", " ", token) 
      token = re.sub(r"\!+","!", token)
      token = re.sub(r"\؟+","؟", token)
      token = re.sub(r"\u200c", " ", token)
      token = token.lower()
      token = re.sub('<[^>]*>', '', token)
      token = re.sub('\S*@\S*\s?', '', token)
      token = re.sub('https?://[A-Za-z0-9]','',token)
      token = re.sub('[^a-zA-Z]',' ',token)
      token = re.sub(r"\'s", "", token)
      token = re.sub(r"\'ve", "", token)
      token = re.sub(r"n\'t", "", token)
      token = re.sub(r"\'re", "", token)
      token = re.sub(r"\'d", "", token)
      token = re.sub(r"\'11", "", token)
      token = re.sub(r"'", "", token)
      token = re.sub(r"\s{2,}", "", token)
      token = re.sub(r"\\n","", token)
      token = re.sub(r"\W","", token)
      token = re.sub(r"https\s+|www.\s+",r"", token)
      token = re.sub(r"won\'t", "will not", token)
      token = re.sub(r"don\'t", "do not", token)
      tokens.append(token)

    tokens = [w for w in tokens if not len(w) <= 1]  
    tokens = [w for w in tokens if not w.isdigit()] 
    tokens = [lemmatizer.lemmatize(w) for w in tokens] 
    tokens = [stemmer.stem(w) for w in tokens] 
    tokens = ' '.join(tokens)
    return tokens

In [ ]:
train_docs = np.empty_like(sentence_train)
for index, document in enumerate(sentence_train):
  train_docs[index] = clean_data(document)

In [ ]:
num_words = 6896
# Create the tokenizer
tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(train_docs)
encoded_docs = tokenizer.texts_to_sequences(train_docs)
print(encoded_docs)

[[5073, 441, 2411, 3997, 1, 10], [359, 195, 359, 195, 359, 195, 719, 590, 79, 2, 1027, 813, 720, 148, 912, 359, 195, 359, 195, 359, 195, 359, 195, 359, 195, 719, 538, 814, 1443, 210, 1166, 172, 1, 3998, 1028, 1, 21, 472, 3999, 2131, 865, 1327, 570, 5074, 5075, 5076, 5077, 3254, 63], [76], [292, 249, 4, 2786, 2787, 12, 2787, 119, 721, 2787, 591, 591, 5078, 5078, 591, 591, 5079, 305, 1237, 968, 325, 111, 1238, 25, 300, 2132, 229, 4, 571, 111, 245, 1167, 111, 37, 59, 111, 86], [1444, 22, 1, 2133, 866, 9, 780, 694, 1444, 22, 1, 2133, 866, 1, 1728, 26, 2134, 572, 1921, 4000, 1592, 1922, 554, 146, 163, 141, 72, 1593, 456, 1168, 695, 5080, 51, 2412, 245, 1729, 815, 125, 1444, 22, 1, 753, 2, 1, 3255], [], [], [2135, 867], [5081], [164, 164, 164, 164, 164], [1, 1730, 1088, 1, 1730, 1088, 1, 1730, 1088, 1, 1730, 1088, 722, 3256, 816, 3256, 2413, 1328], [2414, 2414, 2414, 2414, 2414, 2414], [34, 34, 34, 34, 196, 34, 196, 34, 5082, 8, 34, 34, 34, 34, 34, 81, 34, 86], [969, 21, 60, 654, 1, 1329, 52

In [ ]:
max_length = max([len(s.split()) for s in train_docs])
max_length

1097

In [ ]:
x_train_padded = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
x_train_padded[1]

array([359, 195, 359, ...,   0,   0,   0], dtype=int32)

In [ ]:
def regular_encode(texts, tokenizer, maxlen = 512):
    enc_di = tokenizer.batch_encode_plus(texts, return_attention_masks = False, return_token_type_ids = False, pad_to_max_length = True, max_length = maxlen)
    return np.array(enc_di['input_ids'])

In [ ]:
tokenizer = transformers.BertTokenizer.from_pretrained('bert-large-uncased')

In [ ]:
X_train, y_train = train_test_split(train['label_id'], random_state = 22, test_size = 0.3)
X_test, y_test = train_test_split(train['label_id'], random_state = 22, test_size = 0.3)

In [ ]:
Xtrain_encoded = regular_encode(X_train.astype('str'), tokenizer, maxlen = 128)
ytrain_encoded = tf.keras.utils.to_categorical(y_train, num_classes = 8, dtype = 'int32')
Xtest_encoded = regular_encode(X_test.astype('str'), tokenizer, maxlen = 128)
ytest_encoded = tf.keras.utils.to_categorical(y_test, num_classes = 8, dtype = 'int32')

Streaming output truncated to the last 5000 lines.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognized.
Keyword arguments {'return_attention_masks': False} not recognize

In [ ]:
def build_model(transformer, loss = 'categorical_crossentropy', max_len = 1097):
    input_word_ids = tf.keras.layers.Input(shape = (max_len,), dtype = tf.int32, name = "input_word_ids")
    sequence_output = transformer(input_word_ids)[0]
    cls_token = sequence_output[:, 0, :]
    x = tf.keras.layers.Dropout(0.40)(cls_token)
    out = tf.keras.layers.Dense(3, activation = 'sigmoid')(x)
    model = tf.keras.Model(inputs = input_word_ids, outputs = out)
    model.compile(tf.keras.optimizers.Adam(lr = 3e-5), loss = loss, metrics = ['accuracy'])
    return model

In [ ]:
with strategy.scope():
    transformer_layer = transformers.TFAutoModel.from_pretrained('bert-large-uncased')
    model = build_model(transformer_layer, max_len = 128)
model.summary()


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_word_ids (InputLayer)  [(None, 128)]             0         
_________________________________________________________________
tf_bert_model (TFBertModel)  ((None, 128, 1024), (None 335141888 
_________________________________________________________________
tf_op_layer_strided_slice (T [(None, 1024)]            0         
_________________________________________________________________
dropout_73 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 3)                 3075      
Total params: 335,144,963
Trainable params: 335,144,963
Non-trainable params: 0
_________________________________________________________________


In [ ]:
BATCH_SIZE = 32*strategy.num_replicas_in_sync
AUTO = tf.data.experimental.AUTOTUNE 
train_dataset = (tf.data.Dataset.from_tensor_slices((Xtrain_encoded, ytrain_encoded)).repeat().shuffle(2048).batch(BATCH_SIZE).prefetch(AUTO))
test_dataset = (tf.data.Dataset.from_tensor_slices(Xtest_encoded).batch(BATCH_SIZE))

In [ ]:
n_steps = Xtrain_encoded.shape[0] // BATCH_SIZE
train_history = model.fit(train_dataset, steps_per_epoch = n_steps, epochs = 20)

Epoch 1/20
164/164 [==============================] - 71s 432ms/step - loss: 0.5825 - accuracy: 0.7269
Epoch 2/20
164/164 [==============================] - 71s 431ms/step - loss: 0.3186 - accuracy: 0.8684
Epoch 3/20
164/164 [==============================] - 71s 432ms/step - loss: 0.2723 - accuracy: 0.8886
Epoch 4/20
164/164 [==============================] - 71s 432ms/step - loss: 0.2421 - accuracy: 0.9011
Epoch 5/20
164/164 [==============================] - 71s 431ms/step - loss: 0.1930 - accuracy: 0.9211
Epoch 6/20
164/164 [==============================] - 71s 431ms/step - loss: 0.1592 - accuracy: 0.9361
Epoch 7/20
164/164 [==============================] - 71s 432ms/step - loss: 0.1174 - accuracy: 0.9542
Epoch 8/20
164/164 [==============================] - 71s 431ms/step - loss: 0.0876 - accuracy: 0.9659
Epoch 9/20
164/164 [==============================] - 71s 432ms/step - loss: 0.0641 - accuracy: 0.9767
Epoch 10/20
164/164 [==============================] - 71s 431ms/step - l

In [ ]:
preds = model.predict(test_dataset, verbose = 1)
pred_classes = np.argmax(preds, axis = 1)

71/71 [==============================] - 20s 278ms/step


In [ ]:
print('Prediction Accuracy on Validation dataset: ', np.round(100*accuracy_score(pred_classes, y_test), 8), '%')

Prediction Accuracy on Validation dataset:  88.21 %


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
